# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.381942     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.473908     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [10]:
# model_all.coefficients.print_rows(18)
model_all.coefficients[model_all.coefficients['value'] != 0][['name', 'value']]

name,value
(intercept),274873.0559504957
bathrooms,8468.531086910072
sqft_living,24.420720982445214
sqft_living_sqrt,350.0605533860648
grade,842.0680348976282
sqft_above,20.024722417091304


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [11]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [12]:
import numpy as np
l1_penalty = np.logspace(1, 7, num = 13)

RSS = [0] * 13
for i in range(0, len(l1_penalty)):
   
    model_i = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_penalty[i], verbose = False)
    
    RSS[i] = ((model_i.predict(validation) - validation['price'])**2).sum()
    

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [13]:
RSS[RSS.index(min(RSS))]

625766285142459.9

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [15]:
model_best = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_penalty[0], verbose = False)
# # nonzero weights
model_best.coefficients['value'].nnz()

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [16]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [17]:
l1_penalty_values = np.logspace(8, 10, num=20)


Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [18]:
nnz_coeficients = []
for l1_penalty in l1_penalty_values:
    model_i = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set = None, l2_penalty=0., 
                                                  l1_penalty = l1_penalty, verbose = False)
    
    nnz_coeficients.append((model_i.coefficients['value'].nnz(), l1_penalty))

# exploring the nnz_coeficients list
nnz_coeficients

[(18, 100000000.0),
 (18, 127427498.57031322),
 (18, 162377673.91887242),
 (18, 206913808.111479),
 (17, 263665089.87303555),
 (17, 335981828.6283788),
 (17, 428133239.8719396),
 (17, 545559478.1168514),
 (17, 695192796.1775591),
 (16, 885866790.4100832),
 (15, 1128837891.6846883),
 (15, 1438449888.2876658),
 (13, 1832980710.8324375),
 (12, 2335721469.0901213),
 (10, 2976351441.6313133),
 (6, 3792690190.7322536),
 (5, 4832930238.571753),
 (3, 6158482110.6602545),
 (1, 7847599703.514623),
 (1, 10000000000.0)]

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [19]:
l1_penalty_min = l1_penalty_values[14]
l1_penalty_max = l1_penalty_values[15]
print(l1_penalty_min)
print(l1_penalty_max)


2976351441.6313133
3792690190.7322536


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [20]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [21]:
RSS = [0]*20
model_i = [0]*20

for i in range(0, len(l1_penalty_values)):
   
    model_i[i] = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_penalty_values[i], verbose = False)
    
    RSS[i] = ((model_i[i].predict(validation) - validation['price'])**2).sum()

min_RSS = RSS.index(min(RSS))
min_model_i = model_i[min_RSS].coefficients
print(min_RSS)
print(min_model_i)

0
+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 196100.9378056226  |  None  |
|     bedrooms     |  None | 2181.574321070697  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 17962.696661157814 |  None  |
|   sqft_living    |  None | 34.14246565122554  |  None  |
| sqft_living_sqrt |  None |  789.319789078241  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None | 3665.930817596536  |  None  |
|  floors_square   |  None |        0.0         |  None  |
+------------------+-------+--------------------+--------+
[18 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [22]:
min_RSS_list = []
for i in range(0,20):
    if (model_i[i].coefficients['value'].nnz() == 7 ):
        min_RSS_list.append((l1_penalty_values[i],RSS[i],i))
        
        
    
x = sorted(min_RSS_list)
x

[(3448968612.163437, 1046937488751711.1, 11),
 (3491933809.484539, 1051147625612860.9, 12),
 (3534899006.805641, 1055992735342999.1, 13),
 (3577864204.126743, 1060799531763287.9, 14)]

In [24]:
model_i[11].coefficients[model_i[11].coefficients['value'] != 0][['name','value']]

name,value
(intercept),222253.19254432785
bedrooms,661.7227177822587
bathrooms,15873.957259267981
sqft_living,32.41022145125964
sqft_living_sqrt,690.1147733133256
grade,2899.4202697498786
sqft_above,30.011575302201045
